In [1]:
import numpy as np
import openmm as mm
from openmm import app
import pdbfixer

In [2]:
# define constants and parameters


#molecule
mu_0 = 0.002 #rescaled monomer mobility
l_B =  0.7*mm.unit.angstrom*10 # nanometer
a_beed = 1*mm.unit.angstrom*10
q_beed = 1*mm.unit.elementary_charge
Xi = 6*q_beed
m_C = 12.01078 #g/mol
m_atom = m_C*1e-3*6.022e-23 #kg/particle
gamma = 953975.2938283861 # 1/s friction coefficient, chosen so T=300K
gamma_ps = gamma*1e-12  # 1/ps

chain_beed_distance = 2 # unit less
chain_n_beeds = 2

#unit cell
uc_a = 1.5*chain_beed_distance*chain_n_beeds
uc = (uc_a, uc_a, uc_a, 90, 90, 90)

#environment
# set temperature so the total so the random force is scaled approriately
kB = 1.38e-23

def T_reduced(m, gamma):
    """
    calculates temperature (K) dependent on per particle mass and friction coefficient
    for proper random force scaling
    """
    return 0.006*m*gamma/1.38e-23

temperature = T_reduced(m_atom, gamma)
print("T =", temperature)

print(1/(gamma*m_atom))



T = 300.0
1.4492753623188403e+18


In [3]:
#chain_n_beeds = 5
#chain_beed_distance = 2
file_name = f"polymer_{chain_n_beeds:d}_beeds"

# create pdb file
pdb_file = file_name+".pdb"

# TODO problems:
#   add atom names
# specify bonds using CONECT

with open(pdb_file, "w") as f:
    f.write("HEADER\t"+file_name+"\n")
    f.write(f"CRYST1   60.000   60.000   60.000  90.00  90.00  90.00 P 1           1 \n")
    
    # create chain along the x-axis
    for k in range(chain_n_beeds):
        #f.write(f"HETATM{k+1:5d}	 CA	 HET X       {k*chain_beed_distance+chain_beed_distance:6.3f}   0       0  1.00  0.00          Ca  \n")
        f.write(f"HETATM{k+1:5d} CA   HET X{k+1:4d}    {k*chain_beed_distance+chain_beed_distance:8.3f}{0.0:8.3f}{0.0:8.3f}{1.0:6.2f}{0.0:6.2f}           C  \n")
    #terminate chain
    f.write(f"TER    {k+2:4d}      HET X {k+1:3d}\n")
    
    # add bonds
    f.write(f"CONECT{1:5d}{2:5d}\n") #first beed
    for k in range(2, chain_n_beeds):
        f.write(f"CONECT{k:5d}{k-1:5d}{k+1:5d}\n") #middle beeds
    f.write(f"CONECT{chain_n_beeds:5d}{chain_n_beeds-1:5d}\n") #last beed
    
    f.write("END\n")
    f.close()

In [5]:
#define topology

# TODO make FF paremeters variable
# also: what is my charge?

# create force field in xml file

xml_file = file_name+".xml"
with open(xml_file, "w") as f:
    #initiate
    f.write("<ForceField>\n")
    
    #define atom types
    f.write("\t<AtomTypes>\n\t\t<Type name=\"0\" class=\"CA\" element=\"C\" mass=\"12.01078\"/>\n\t</AtomTypes>\n")
    
    #define residue

    #add atoms
    f.write("\t<Residues>\n\t\t<Residue name=\"HET\">\n")
    for k in range(1, chain_n_beeds+1):
        f.write(f"\t\t\t<Atom name=\"{k:d}\" type=\"0\"/>\n")
        
    #add bonds
    f.write(f"\t\t\t<Bond atomName1=\"{1:d}\" atomName2=\"{2:d}\"/>\n")
    for k in range(2, chain_n_beeds):
        f.write(f"\t\t\t<Bond atomName1=\"{k:d}\" atomName2=\"{k-1:d}\"/>\n")
        f.write(f"\t\t\t<Bond atomName1=\"{k:d}\" atomName2=\"{k+1:d}\"/>\n")
    f.write(f"\t\t\t<Bond atomName1=\"{chain_n_beeds:d}\" atomName2=\"{chain_n_beeds-1:d}\"/>\n")
    f.write("\t\t</Residue>\n\t</Residues>\n")
    
    #define forcefield
    
    #add nearest neighbours
    f.write("\t<CustomBondForce energy=\"K*(r-r0)^2\">\n")
    f.write("\t\t<GlobalParameter name=\"K\" defaultValue=\"100\"/>\n")
    f.write("\t\t<GlobalParameter name=\"r0\" defaultValue=\"2\"/>\n")
    f.write("\t\t<Bond class1=\"CA\" class2=\"CA\"/>\n")
    f.write("\t</CustomBondForce>\n")
    
    # TODO: CHOOSE SENSIBLE CUTOFF 
    
    #add debye interaction
    f.write("\t<CustomNonbondedForce energy=\"q^2*lB*exp(r)/r\" bondCutoff=\"4\">\n")
    f.write("\t\t<GlobalParameter name=\"lB\" defaultValue=\"4\"/>\n")
    f.write("\t\t<GlobalParameter name=\"q\" defaultValue=\"1\"/>\n")
    f.write("\t\t<Atom type=\"0\"/>\n")
    f.write("\t</CustomNonbondedForce>\n")
    
    #add LJ interaction
    # NOTE is the +1 in the LJ dependant on the units???
    # TODO: make cutoff and sigma variable
    f.write("\t<CustomNonbondedForce energy=\"epsilon*(2^12/r^12 - 2^7/r^6 + 1)\" bondCutoff=\"2\">\n")
    f.write("\t\t<GlobalParameter name=\"epsilon\" defaultValue=\"1\"/>\n")
    f.write("\t\t<Atom type=\"0\"/>\n")
    f.write("\t</CustomNonbondedForce>\n")
    
    #close xml
    f.write("</ForceField>\n")
    f.close()

In [6]:
# create system
pdb = app.pdbfile.PDBFile(pdb_file)

forcefield = app.ForceField(xml_file)
system = forcefield.createSystem(pdb.topology)

# TODO: choose good timestep and timescale
timestep = 0.001

"""The Brownian equation of motion is derived from the Langevin equation of motion 
in the limit of large gamma In that case, the velocity of a particle is determined 
entirely by the instantaneous force acting on it, and kinetic energy ceases to have 
much meaning, since it disappears as soon as the applied force is removed."""


# define integrator 
# NOTE: for this simulation the friction is set to zero
# set temperature so the total so the random force is scaled approriately
integrator = mm.openmm.BrownianIntegrator(temperature*mm.unit.kelvin, gamma/mm.unit.picoseconds, timestep*mm.unit.picoseconds)

    
# prepare simulation
platform = mm.Platform.getPlatformByName('CPU') # change from CUDA to CPU

# NOTE: the integrator must be defined to set up the simulation class, but it is not used
simulation = app.simulation.Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.positions)# Set initial positions

# integrate manually and use openmm force calculations
state = simulation.context.getState(getPositons=True, getVelocities=True, getForces=True)
positions = state.getPositons(asNumpy=True)
forces = state.getForces(asNumpy=True)
velocities = state.getVelocities(asNumpy=True)
# integration routine with numpy arrays
simulation.context.setPositions(positions)
simulation.context.setVelocities(velocities)

# minimize energy of the system to avoid any possible overlaps that may result in high forces
# (not necessary here, but still)
simulation.minimizeEnergy()


# simulate

ValueError: No template found for residue 1 (HET).  This might mean your input topology is missing some atoms or bonds, or possibly that you are using the wrong force field.

<function openmm.openmm.LangevinIntegrator.__init__(self, *args)>